In [1]:
from threading import Thread

import paho.mqtt.client as mqtt


class MQTT_Client_1:
    def __init__(self):
        self.client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message

    def on_connect(self, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, msg):
        print("on_message(): topic: {}".format(msg.topic))
        self.stm_driver.send("message", "tick_tock")

    def start(self, broker, port):

        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)

        self.client.subscribe("eirikh_from_scu")

        try:
            # line below should not have the () after the function!
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()

In [2]:
from stmpy import Machine, Driver

import ipywidgets as widgets
from IPython.display import display

def on_connect(rc):
    print("Connected with result code "+str(rc))

# The callback for when a PUBLISH message is received from the server.
def on_message(msg):
    print(msg.topic+" "+str(msg.payload))

myclient = MQTT_Client_1()
myclient.on_connect = on_connect
myclient.on_message = on_message

broker, port = "test.mosquitto.org", 1883

class Charger:
    
    def on_buttonConnect_press(self, b):
        self.stm.send('car_connected') # <---- here we send a signal
    
    def on_buttonCharge_press(self, b):
        self.stm.send('request_charging') # <---- here we send a signal
    
    def on_buttonDisconnect_press(self, b):
        self.stm.send('car_disconnected') # <---- here we send a signal

    def on_buttonChargingFinished_press(self, b):
        self.stm.send('charging_finished') # <---- here we send a signal
    
    def on_buttonChargingAborted_press(self, b):
        self.stm.send('charging_aborted') # <---- here we send a signal

    def update_scu(self):
        print("Updating SCU")
        self.mqtt_client.publish("eirikh", "Car connected")
    
    def charge_car(self):
        print("Charging car")
            
    def __init__(self):
        # load images and store them
        self.buttonConnect = widgets.Button(description="Connect car")
        self.buttonConnect.on_click(self.on_buttonConnect_press)
        
        self.buttonCharge = widgets.Button(description="Charge")
        self.buttonCharge.on_click(self.on_buttonCharge_press)

        self.buttonDisconnect = widgets.Button(description="Disconnect car")
        self.buttonDisconnect.on_click(self.on_buttonDisconnect_press)

        self.buttonChargingFinished = widgets.Button(description="Finish charging")
        self.buttonChargingFinished.on_click(self.on_buttonChargingFinished_press)

        self.buttonChargingAborted = widgets.Button(description="Abort charging")
        self.buttonChargingAborted.on_click(self.on_buttonChargingAborted_press)

        display(self.buttonConnect, self.buttonCharge, self.buttonDisconnect, self.buttonChargingFinished, self.buttonChargingAborted)

/var/folders/zm/t0k9nnts7jbfkwkm5lg1c_5r0000gn/T/ipykernel_25570/672193713.py:8: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  self.client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)


In [3]:
charger = Charger()

t0_initial = {'source': 'initial',
      'target': 'idle'}

t1_car_connected = {'trigger': 'car_connected',
        'source': 'idle',
        'target': 'connected'
}

t2_request_charging = {'trigger': 'request_charging',
        'source': 'connected',
        'target': 'charging'
}
t3_car_disconnected = {'trigger': 'car_disconnected',
        'source': 'connected',
        'target': 'idle'
}
t4_charging_finished = {'trigger': 'charging_finished',
        'source': 'charging',
        'target': 'connected'
}
t5_charging_aborted = {'trigger': 'charging_aborted',
        'source': 'charging',
        'target': 'connected'
}

idle = {'name': 'idle'}

connected = {'name': 'connected',
             'entry': 'update_scu'}

charging = {'name': 'charging',
            'entry': 'charge_car'}

machine = Machine(name='charger', transitions=[t0_initial, t1_car_connected, t2_request_charging, t3_car_disconnected, t4_charging_finished, t5_charging_aborted],
                                states=[idle, connected, charging], obj=charger)

charger.stm = machine
driver = Driver()
driver.add_machine(machine)
driver.start()
charger.mqtt_client = myclient.client
myclient.start(broker, port)

Button(description='Connect car', style=ButtonStyle())

Button(description='Charge', style=ButtonStyle())

Button(description='Disconnect car', style=ButtonStyle())

Button(description='Finish charging', style=ButtonStyle())

Button(description='Abort charging', style=ButtonStyle())

Connecting to test.mosquitto.org:1883


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/eirikhoem/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/eirikhoem/Library/Python/3.9/lib/python/site-packages/paho/mqtt/client.py", line 2291, in loop_forever
    rc = self._loop(timeout)
  File "/Users/eirikhoem/Library/Python/3.9/lib/python/site-packages/paho/mqtt/client.py", line 1680, in _loop
    rc = self.loop_read()
  File "/Users/eirikhoem/Library/Python/3.9/lib/python/site-packages/paho/mqtt/client.py", line 2094, in loop_read
    rc = self._packet_read()
  F